# Identifying signal components across multiple scales of network topology

This demo shows you how to use methods in `graspologic` to analyze patterns
in brain connectivity in connectomics datasets. We specifically demonstrate
methods for identifying differences in edges and vertices across subjects. 

This notebook replicates Figure 4 of _Multiscale Comparative Connectomics_.
![Fig4](figures/4_signal_tractograms.jpg)

Note that tractograms are not produced in this notebook.
Tractograms of various signal components were made using DSI Studio.
Tracking parameters used in DSI Studio are available in `supplement/tracking_parameters`
and the necessary MRI data can be requested from Dr. G. Allan Johnson.

In [1]:
%load_ext nb_black
%load_ext rpy2.ipython

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from graspologic.datasets import load_mice
from statsmodels.stats.multitest import multipletests

## Load the Duke mouse brain dataset

Dataset of 32 mouse connectomes derived from whole-brain diffusion
magnetic resonance imaging of four distinct mouse genotypes:
BTBR T+ Itpr3tf/J (BTBR), C57BL/6J(B6), CAST/EiJ (CAST), and DBA/2J (DBA2).
For each strain, connectomes were generated from eight age-matched mice
(N = 8 per strain), with a sex distribution of four males and four females.
Each connectome was parcellated using asymmetric Waxholm Space, yielding a
vertex set with a total of 332 regions of interest (ROIs) symmetrically
distributed across the left and right hemispheres. Within a given
hemisphere, there are seven superstructures consisting up multiple ROIs,
resulting in a total of 14 distinct communities in each connectome.

In [3]:
# Load the full mouse dataset
mice = load_mice()

# Stack all adjacency matrices in a 3D numpy array
graphs = np.array(mice.graphs)

# Sort the connectomes and genotype labels so BTBR is first
label_indices = np.argsort(mice.labels).reshape(4, 8)
label_indices = label_indices[[1, 0, 2, 3]].reshape(-1)
labels = mice.labels[label_indices]
graphs = graphs[label_indices]

# Get sample parameters
n_subjects = mice.meta["n_subjects"]
n_vertices = mice.meta["n_vertices"]

## Identifying Signal Edges

The simplest approach for comparing connectomes is to treat them as a _bag of edges_ without considering interactions between the edges.
Serially performing univariate statistical analyses at each edge enables the discovery of _signal edges_ whose neurological connectivity differs across categorical or dimensional phenotypes.
Here, we demonstrate the possibility of using Distance Correlation (`Dcorr`), a nonparametric universally consistent test, to detect changes in edges.

In this model, we assume that each edge in the connectome is independently and
identically sampled from some distribution $F_i$, where $i$ represents the group
to which the given connectome belongs. In this setting the groups are the mouse
genotypes. `Dcorr` allows us to test the following hypothesis:

\begin{align*}
H_0:\ &F_1 = F_2 = \ldots F_k \\
H_A:\ &\exists \ j \neq j' \text{ s.t. } F_j \neq F_{j'}
\end{align*}

In [4]:
from hyppo.ksample import KSample

In [5]:
# Split the set of graphs by genotype
btbr = graphs[labels == "BTBR"]
b6 = graphs[labels == "B6"]
cast = graphs[labels == "CAST"]
dba2 = graphs[labels == "DBA2"]

connectomes = [btbr, b6, cast, dba2]

Since the connectomes in this dataset are undirected, we only need to do edge
comparisons on the upper triangle of the adjacency matrices.

In [6]:
# Make iterator for traversing the upper triangle of the connectome
indices = zip(*np.triu_indices(n_vertices, 1))

In [7]:
edge_pvals = []

for roi_i, roi_j in indices:

    # Get the (i,j)-th edge for each connectome
    samples = [genotype[:, roi_i, roi_j] for genotype in connectomes]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    edge_pvals.append([roi_i + 1, roi_j + 1, statistic, pvalue])

Connectomes are a high-dimensional dataype.
Thus, statistical tests on components of the connectome (e.g. edge and vertices)
results in multiple comparisons.
We recommend correcting for multiple comparisons using the Holm-Bonferroni (HB)
corection.

The algorithm is described below:
1. Sort the p-values from lowest-to-highest $P_1, P_2, \dots, P_n$, where $n$ is the number of tests
2. Correct the p-value as $P_1(n), P_2(n-1), \dots, P_n(1)$
3. If any corrected p-value is $>1$, replace with $1$
4. If the corrected p-value is less than a significance level $\alpha$, reject

In [8]:
# Convert the nested list to a dataframe
signal_edges = pd.DataFrame(edge_pvals, columns=["ROI_1", "ROI_2", "stat", "pvalue"])
signal_edges.sort_values(by="pvalue", inplace=True, ignore_index=True)

# Correct p-values
reject, holm_pvalue, _, _ = multipletests(
    signal_edges["pvalue"], alpha=0.05, method="holm"
)
signal_edges["holm_pvalue"] = holm_pvalue
signal_edges["significant"] = reject
signal_edges.sort_values(by="holm_pvalue", inplace=True, ignore_index=True)
signal_edges.to_csv("../results/signal_edges.csv", index=False)
signal_edges.head()

In [9]:
def lookup_roi_name(roi):
    roi -= 1
    hemisphere = "R" if roi // 166 else "L"
    roi = roi % 166
    structure = mice.atlas["Structure"].values[roi]
    structure = " ".join(structure.split("_"))
    return f"{structure} ({hemisphere})"

In [10]:
# Get the top 20 strongest signal edges
strong_signal_edges = signal_edges.head(20)
strong_signal_edges["ROI_1"] = strong_signal_edges["ROI_1"].apply(lookup_roi_name)
strong_signal_edges["ROI_2"] = strong_signal_edges["ROI_2"].apply(lookup_roi_name)
strong_signal_edges

Note that none of the edges achieve significance at $\alpha=0.05$ following 
Holm-Bonferroni correction.
We might expect this, given that we are correcting for $N=54,946$ tests.
Instead of the magnitude, the **ranking of the p-values** can be used to determine signal edges.

In [ ]:
def stripplot(df, data_column):

    kwargs = {
        "alpha": 0.75,
        "edgecolor": None,
        "linewidth": 0,
        "marker": "o",
        "palette": ["#e7298a", "#1b9e77", "#d95f02", "#7570b3"],
    }

    fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=150)

    g = sns.stripplot(
        x="Strain",
        y=data_column,
        data=df,
        jitter=True,
        orient="v",
        ax=ax,
        **kwargs,
    )

    return g

Plot the distribution of edgeweights for the top signal edge across all genotypes.
Clearly different!

In [11]:
edgeweight = graphs[:, 121 - 1, 230 - 1]
edgeweight = pd.DataFrame({"Edgeweight": edgeweight, "Strain": labels})
g = stripplot(edgeweight, "Edgeweight")
g.ticklabel_format(style="sci", scilimits=(0, 0), axis="y")
plt.show()

## Identifying Signal Vertices

A sample of connectomes can be jointly embedded in a low-dimensional Euclidean space using the omnibus embedding (`omni`).
A host of machine learning tasks can be accomplished with this joint embedded representation of the connectome, such as clustering or classification of vertices.
Here, we use the embedding to formulate a statistical test that can be used to identify vertices that are strongly associated with given phenotypes.
According to a Central Limit Theorem for `omni`, these latent positions are universally consistent and asymptotically normal.

In [12]:
from itertools import product

from graspologic.embed import OmnibusEmbed

In [13]:
# Jointly embed graphs using OMNI
embedder = OmnibusEmbed()
omni_embedding = embedder.fit_transform(graphs)
omni_embedding = omni_embedding.reshape(-1, omni_embedding.shape[-1])
print(f"Omnibus embedding shape is {omni_embedding.shape}")

# Convert array to a dataframe
omni_embedding = pd.DataFrame(
    omni_embedding, columns=[f"omni_{i + 1}" for i in range(omni_embedding.shape[-1])]
).astype(np.float64)
omni_embedding.head()

In [14]:
# Construct identifiers for each embedded vertex
left = mice.atlas["ROI"].unique()
right = left + 166
rois = np.append(left, right)

participants = mice.participants["participant_id"]
participants = participants.apply(lambda x: x.split("-")[1])

identifiers = np.array(list(product(participants, rois))).reshape(-1, 2)
identifiers = pd.DataFrame(identifiers, columns=["participant_id", "ROI"])
identifiers["ROI"] = identifiers["ROI"].astype(np.int64)
identifiers["genotype"] = np.array([[strain] * 332 for strain in labels]).reshape(-1)

omni = pd.concat([omni_embedding, identifiers], axis=1)
omni.head()

For each of the 32 mice, `omni` embeds each vertex in the connectome to a
latent position vector $x_i \in \mathbb{R}^5$.
We test for differences in the distribution of vertex latent positions using 
the `R` implementation of `MANOVA`.

In [15]:
%%R -i omni -i n_vertices -o signal_vertices

suppressPackageStartupMessages(require(tidyverse))

col1 <- which(grepl("omni", names(omni))) # column indices for the embeddings
col2 <- which(grepl("genotype", names(omni))) # column index for the genotype

embedding <- colnames(omni)[col1]
genotype <- colnames(omni)[col2]
form <- paste0("cbind(", paste(embedding, collapse=", "), ") ~ ", genotype)

pvec <- rep(0, n_vertices)
pillai <- rep(0, n_vertices)
F <- rep(0, n_vertices)
num.df <- rep(0, n_vertices)
den.df <- rep(0, n_vertices)

for (i in 1 : n_vertices) {
    omni.v <- omni[which(omni$ROI == i), ]
    ans <- manova(as.formula(form), data=omni.v)
    pvec[i] <- summary(ans)$stats[1, "Pr(>F)"]
    pillai[i] <- summary(ans)$stats[1, "Pillai"]
    F[i] <- summary(ans)$stats[1, "approx F"]
    num.df[i] <- summary(ans)$stats[1, "num Df"]
    den.df[i] <- summary(ans)$stats[1, "den Df"]
}

signal_vertices <- data.frame(ROI=unique(omni$ROI), pillai=pillai, F=F, num.df=num.df, den.df=den.df, pvalue=pvec)

In [16]:
# Correct p-values
signal_vertices.sort_values(by="pvalue", inplace=True, ignore_index=True)
reject, holm_pvalue, _, _ = multipletests(
    signal_vertices["pvalue"], alpha=0.05, method="holm"
)
signal_vertices["holm_pvalue"] = holm_pvalue
signal_vertices["significant"] = reject
signal_vertices.sort_values(by="holm_pvalue", inplace=True, ignore_index=True)
signal_vertices.to_csv("../results/signal_vertices.csv", index=False)

In [17]:
# Get the top 10 strongest signal edges
strong_signal_vertices = signal_vertices.head(10)
strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)
strong_signal_vertices

The strongest signal vertex is the left hemisphere corpus callosum.
THe corpus callosum is the bridge between the left and right hemispheres of the brain.
In BTBR mice, this connection is absent (see the tractograms above).
We would expect our methods to recover this vertex as highly heterogeneous
across strains. This experiment validates this hypothesis!

In [18]:
left_cc = omni.query("ROI == 121")[["omni_1", "genotype"]]
left_cc.columns = ["Embedding Dim. 1", "Strain"]
g = stripplot(left_cc, "Embedding Dim. 1")
plt.show()

## Identifying Signal Communities
Communities are the connections between distinct superstructures in the brain (effectively subgraphs of the connectome).
To test for differences in these subgraphs, we use `DCorr`.

In [19]:
from collections import namedtuple
from itertools import combinations_with_replacement

Point = namedtuple("Point", ["x", "y"])

In [20]:
def _get_point(community, hemisphere):
    """Make points from database queries."""
    expr = f"block == '{community}' and hemisphere == '{hemisphere}'"
    point = Point(*mice.blocks.query(expr).values[0][2:])
    return point

In [21]:
def _get_edges(point_1, point_2, sample):
    return sample[:, point_1.x : point_1.y, point_2.x : point_2.y].reshape(8, -1)

In [22]:
def _get_community_name(block, hemisphere):
    block = " ".join([struct.capitalize() for struct in block.split("_")])
    return f"{block} ({hemisphere})"

In [23]:
signal_communities = []

for (_, community_1), (_, community_2) in combinations_with_replacement(
    mice.blocks.iterrows(), r=2
):

    point_1 = _get_point(community_1.block, community_1.hemisphere)
    point_2 = _get_point(community_2.block, community_2.hemisphere)

    community_1 = _get_community_name(community_1.block, community_1.hemisphere)
    community_2 = _get_community_name(community_2.block, community_2.hemisphere)

    edges = [_get_edges(point_1, point_2, sample) for sample in connectomes]

    try:
        stat, pvalue = KSample("Dcorr").test(*edges)
    except ValueError as e:
        print(e)
        stat, pvalue = np.nan, 1

    signal_communities.append([community_1, community_2, stat, pvalue])

In [24]:
signal_communities = pd.DataFrame(
    signal_communities, columns=["Community 1", "Community 2", "statistic", "pvalue"]
)

# Correct p-values
signal_communities = signal_communities.sort_values(["pvalue"])
reject, holm_pvalue, _, _ = multipletests(
    signal_communities["pvalue"], alpha=0.05, method="holm"
)
signal_communities["holm_pvalue"] = holm_pvalue
signal_communities["significant"] = reject
signal_communities.head()

In [25]:
signal_communities.to_csv("../results/signal_communities.csv", index=False)

In [26]:
wm_r = _get_point("white_matter", "R")
wm_r_edgeweight = graphs[:, wm_r.x : wm_r.y, wm_r.x : wm_r.y].reshape(32, -1)
wm_r_mean = wm_r_edgeweight.mean(axis=1)

avg_connectivity = pd.DataFrame({"Mean Edgeweight": wm_r_mean, "Strain": labels})
g = stripplot(avg_connectivity, "Mean Edgeweight")
plt.show()